In [ ]:
%env OPENAI_API_KEY="keyhere"

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools.argo import (
    # Rollout Management
    CreateRolloutResource,
    ListRollouts,
    GetRollout,
    PauseRollout,
    PromoteRollout,
    SetRolloutImage,
    StatusRollout,
    # Installation & Verification
    VerifyArgoRolloutsControllerInstall,
    VerifyKubectlPluginInstall,
    CheckPluginLogsTool,
    VerifyGatewayPluginTool,
    # Resource Generation
    GenerateResource,
)

from kagent.tools.k8s import (
    ApplyManifest,
    GetPods,
    GetResources,
    GetServices,
)

from kagent.tools.docs import Config, QueryTool

# Model configuration
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

# Documentation agent configuration
docs_config = Config(
    docs_path="",  # empty string means the database will be downloaded
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks that can be executed by the team members. DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    Your team members are:
        istio_agent: Performs Istio resource generation tasks.
        k8s_agent: Performs Kubernetes tasks.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

# Define tool groups for better organization
k8s_tools = [GetPods(), GetServices(), GetResources(), ApplyManifest()]

argo_management_tools = [
    CreateRolloutResource(),
    ListRollouts(),
    GetRollout(),
    PauseRollout(),
    PromoteRollout(),
    SetRolloutImage(),
    StatusRollout(),
]

argo_setup_tools = [
    VerifyKubectlPluginInstall(),
    VerifyArgoRolloutsControllerInstall(),
    CheckPluginLogsTool(),
    VerifyGatewayPluginTool(),
    GenerateResource(),
]

K8S_SYSTEM_MESSAGE = """You are a Kubernetes specialist agent responsible for cluster operations and resource management.

Key Responsibilities:
1. Resource Management
   - Monitor and manage Kubernetes resources (pods, services, deployments)
   - Apply and validate Kubernetes manifests
   - Ensure proper resource state and health

2. Cluster Operations
   - Verify cluster connectivity and health
   - Monitor resource utilization
   - Report on cluster state

Guidelines:
- Always verify resource existence before operations
- Use proper error handling for kubectl operations
- Provide clear feedback on operation status
- If no explicit tasks remain, return TERMINATE

Example interactions:
- "Get pods in namespace": List all pods with their status
- "Apply manifest": Validate and apply provided manifest
- "Check service": Verify service existence and endpoints
"""

ARGO_SYSTEM_MESSAGE = """You are an Argo Rollouts specialist focused on progressive delivery and deployment automation.

Key Responsibilities:
1. Installation & Setup
   - Verify kubectl plugin and controller installation
   - Guide through initial setup process
   - Validate system requirements

2. Rollout Management
   - Create and configure rollouts
   - Manage canary deployments
   - Handle promotion and rollback
   - Monitor rollout status

3. Integration
   - Configure traffic management (Istio/Gateway API)
   - Set up metrics-based analysis
   - Integrate with existing deployment workflows

Best Practices:
- Always verify prerequisites before operations
- Provide step-by-step guidance for complex tasks
- Ensure proper validation at each stage
- Document any configuration changes

Example commands:
- `kubectl argo rollouts get rollout <name>`
- `kubectl argo rollouts promote <name>`
- `kubectl argo rollouts status <name>`
"""

DOCS_SYSTEM_MESSAGE = """You are a CNCF documentation specialist providing accurate information from official sources.

Key Responsibilities:
1. Documentation Search
   - Search across supported products: 'kubernetes', 'istio', 'argo cd', 'argo rollouts', 'helm', 'prometheus'
   - Provide accurate, relevant information
   - Include documentation links

2. Information Analysis
   - Understand context of queries
   - Cross-reference multiple sources
   - Provide comprehensive answers

Guidelines:
- Always verify product context before searching
- Provide direct documentation links
- Explain complex concepts clearly
- Cross-reference related documentation

Process for Each Query:
1. Identify target product
2. Search relevant documentation
3. Analyze and synthesize information
4. Provide clear, documented response
"""

# Create agents with improved configurations
k8s_agent = AssistantAgent(
    "k8s_agent",
    description="Kubernetes specialist for cluster operations and resource management",
    tools=k8s_tools,
    model_client=model_client,
    system_message=K8S_SYSTEM_MESSAGE,
)

argo_agent = AssistantAgent(
    "argo_agent",
    description="Argo Rollouts specialist for progressive delivery management",
    tools=argo_management_tools + argo_setup_tools,
    model_client=model_client,
    system_message=ARGO_SYSTEM_MESSAGE,
)

docs_agent = AssistantAgent(
    "cncf_docs_agent",
    description="CNCF documentation specialist for accurate information retrieval",
    tools=[QueryTool(config=docs_config)],
    model_client=model_client,
    system_message=DOCS_SYSTEM_MESSAGE,
)

# Configure team with improved termination conditions
team = RoundRobinGroupChat(
    max_turns=5,  # Increased from 3 to allow for more complex interactions
    participants=[planning_agent, k8s_agent, argo_agent, docs_agent],
    termination_condition=TextMentionTermination(text="TERMINATE"),
)


# Example usage
async def run_team_task(task: str):
    """Run a task through the multi-agent team with proper error handling. If you don't have any explicit tasks left to complete, return TERMINATE."""
    try:
        return await Console(team.run_stream(task=task))
    except Exception as e:
        print(f"Error executing task: {e}")
        return None


# Usage example:
# task = "Help me switch to Argo Rollouts in my cluster?"
# task = "Help me use Kubernetes Gateway APIs with argo rollouts in my cluster?"
task = "Translate my current reviews Kubernetes deployments in the default namespace to an Argo Rollout using the canary strategy."
await run_team_task(task)

---------- user ----------
Translate my current reviews Kubernetes deployments in the default namespace to an Argo Rollout using the canary strategy.
---------- PlanningAgent ----------
To translate the current Kubernetes deployments to an Argo Rollout using the canary strategy, we'll need to undertake the following steps:

1. Gather information about the existing Kubernetes deployments.
2. Convert each Kubernetes deployment to an Argo Rollout resource with a canary strategy.

Here are the tasks to achieve this:

1. k8s_agent: List all Kubernetes deployments in the default namespace.
2. k8s_agent: Retrieve detailed configurations of each deployment in the default namespace.
3. istio_agent: Generate Argo Rollout resource configurations with a canary strategy for each deployment configuration retrieved.
4. k8s_agent: Apply the new Argo Rollout configurations in the default namespace, replacing the existing deployments. 

Please proceed with these tasks to implement the conversion.
------

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Translate my current reviews Kubernetes deployments in the default namespace to an Argo Rollout using the canary strategy.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=164, completion_tokens=158), content="To translate the current Kubernetes deployments to an Argo Rollout using the canary strategy, we'll need to undertake the following steps:\n\n1. Gather information about the existing Kubernetes deployments.\n2. Convert each Kubernetes deployment to an Argo Rollout resource with a canary strategy.\n\nHere are the tasks to achieve this:\n\n1. k8s_agent: List all Kubernetes deployments in the default namespace.\n2. k8s_agent: Retrieve detailed configurations of each deployment in the default namespace.\n3. istio_agent: Generate Argo Rollout resource configurations with a canary strategy for each deployment configuration retrieved.\n4. k8s_agent: Apply the new

In [ ]:
print(team.dump_component().model_dump_json(indent=2))